In [ ]:
## Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from math import sin, cos, sqrt, atan2, radians
import warnings

In [2]:
## load up zip code/geocode data
geoData = pd.read_csv('../Data/zipCodeGeoCode.csv', converters={'Zip':str})

In [3]:
## load up plant data
plantData = pd.read_excel('../Data/LatLongPlants.xlsx', converters={'Zip':str})

In [4]:
geoData.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,05769,Salisbury,VT,43.914026,-73.107030,-5,1,"43.914026, -73.10703"
1,01085,Westfield,MA,42.133642,-72.750290,-5,1,"42.133642, -72.75029"
2,01340,Colrain,MA,42.673371,-72.731040,-5,1,"42.673371, -72.73104"
3,25728,Huntington,WV,38.413384,-82.277401,-5,1,"38.413384, -82.277401"
4,26035,Colliers,WV,40.352943,-80.555850,-5,1,"40.352943, -80.55585"


In [5]:
## load up zip code/geocode data
zipPopAll = pd.read_excel('../Data/2010+Census+Population+By+Zipcode+(ZCTA).xlsx', converters={'Zip Code ZCTA':str})
zipPopAll.head()


,Zip Code ZCTA,2010 Census Population
0,01001,16769.0
1,01002,29049.0
2,01003,10372.0
3,01005,5079.0
4,01007,14649.0


In [6]:
zipPopAll.head()

,Zip Code ZCTA,2010 Census Population
0,01001,16769.0
1,01002,29049.0
2,01003,10372.0
3,01005,5079.0
4,01007,14649.0


In [7]:
len(zipPopAll)

33096

In [8]:
len(geoData)

43191

In [9]:
## find lat/long 
mergeDF = pd.merge(geoData, zipPopAll, left_on='Zip', right_on='Zip Code ZCTA')

In [10]:
mergeDF.head()


,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint,Zip Code ZCTA,2010 Census Population
0,05769,Salisbury,VT,43.914026,-73.10703,-5,1,"43.914026, -73.10703",05769,1216.0
1,01085,Westfield,MA,42.133642,-72.75029,-5,1,"42.133642, -72.75029",01085,41117.0
2,01340,Colrain,MA,42.673371,-72.73104,-5,1,"42.673371, -72.73104",01340,1701.0
3,26035,Colliers,WV,40.352943,-80.55585,-5,1,"40.352943, -80.55585",26035,2491.0
4,36279,Wellington,AL,33.862349,-85.89512,-6,1,"33.862349, -85.89512",36279,2645.0


In [11]:
geoPop = mergeDF[['Zip', 'City', 'State', 'Latitude', 'Longitude', '2010 Census Population']].copy()

In [12]:
def distanceCalculator(lat1Reactor, long1Reactor, lat2City, long2City):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1Reactor)
    lon1 = radians(long1Reactor)
    lat2 = radians(lat2City)
    lon2 = radians(long2City)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance



In [13]:
geoPop.head()


,Zip,City,State,Latitude,Longitude,2010 Census Population
0,05769,Salisbury,VT,43.914026,-73.10703,1216.0
1,01085,Westfield,MA,42.133642,-72.75029,41117.0
2,01340,Colrain,MA,42.673371,-72.73104,1701.0
3,26035,Colliers,WV,40.352943,-80.55585,2491.0
4,36279,Wellington,AL,33.862349,-85.89512,2645.0


In [14]:
#def reactorDistance(popDF, lat1Reactor, long1Reactor):
# for index, row in geoPop.iterrows():
#    #     warnings.filterwarnings('ignore')
#     lat1Reactor = 35.330898
#     long1Reactor = -93.247162
#     lat2City = row['Latitude']
#     long2City = row['Longitude']
#     dist = distanceCalculator(lat1Reactor, long1Reactor, lat2City, long2City)
#     geoPop.loc[index, 'Distance from Reactor'] = dist

    


In [15]:
def reactorDistance(popDF, lat1Reactor, long1Reactor, plantName):
    for index, row in popDF.iterrows():
        lat2City = row['Latitude']
        long2City = row['Longitude']
        dist = distanceCalculator(lat1Reactor, long1Reactor, lat2City, long2City)
        colTitle = f'Distance from {plantName}'
        popDF.loc[index, colTitle] = dist
    
    return popDF

In [16]:
for index, row in plantData.iterrows():
    lat1Reactor = row['Lat']
    long1Reactor = row['Long']
    plantName = row['Name']
#     print(plantName)
    geoPop = reactorDistance(geoPop, lat1Reactor, long1Reactor, plantName)

In [17]:
#plantData.head()
geoPop.head()

,Zip,City,State,Latitude,Longitude,2010 Census Population,"Distance from Arkansas Nuclear One, Unit 1","Distance from Arkansas Nuclear One, Unit 2",Distance from Beaver Valley Power Station 1,Distance from Beaver Valley Power Station 2,...,Distance from Susquehanna 1,Distance from Susquehanna 2,Distance from Turkey Point 3,Distance from Turkey Point 4,Distance from Vogtle 1,Distance from Vogtle 2,Distance from Waterford 3,Distance from Watts Bar 1,Distance from Watts Bar 2,Distance from Wolf Creek 1
0,05769,Salisbury,VT,43.914026,-73.10703,1216.0,1964.516138,1964.516138,703.791569,703.791569,...,655.662689,655.662689,2157.290611,2157.290611,1430.107582,1430.107582,2177.150254,1355.830978,1355.830978,1994.970315
1,01085,Westfield,MA,42.133642,-72.75029,41117.0,1926.478786,1926.478786,661.448479,661.448479,...,635.033811,635.033811,1984.280764,1984.280764,1293.831261,1293.831261,2080.818508,1267.610472,1267.610472,1996.520992
2,01340,Colrain,MA,42.673371,-72.73104,1701.0,1945.702821,1945.702821,678.067625,678.067625,...,644.811576,644.811576,2040.142885,2040.142885,1340.062884,1340.062884,2116.809863,1300.357427,1300.357427,2004.169739
3,26035,Colliers,WV,40.352943,-80.55585,2491.0,1245.246703,1245.246703,32.787622,32.787622,...,113.903325,113.903325,1655.585469,1655.585469,818.234170,818.234170,1460.484573,640.926797,640.926797,1327.964576
4,36279,Wellington,AL,33.862349,-85.89512,2645.0,692.507353,692.507353,894.790543,894.790543,...,966.988675,966.988675,1069.953709,1069.953709,369.996463,369.996463,609.421102,224.390980,224.390980,1007.760155


In [18]:
geoPop.to_csv('../Data/plantPop.csv')

In [19]:
## load up zip code/geocode data
geoDataNoDupes = pd.read_csv('../Data/plantPops_noDuplicates.csv', converters={'Zip':str})

FileNotFoundError: [Errno 2] File ../Data/plantPops_noDuplicates.csv does not exist: '../Data/plantPops_noDuplicates.csv'

In [ ]:
lat1Reactor = 35.330898
long1Reactor = -93.247162

geoPop = reactorDistance(geoPop, lat1Reactor, long1Reactor)

In [ ]:
geoPop.sort_values(by='Distance from Reactor', inplace=True)
geoPop.head()

In [ ]:
plt.plot(geoPop['Distance from Reactor'], geoPop['2010 Census Population'])
        
plt.xlim(0, 500)
plt.xlabel('Distance From Reactor')
plt.ylabel('Population')